In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
L=40
states=np.random.choice([-1,1],size=(10000,L))

In [3]:
print(states.shape)

(10000, 40)


In [4]:
states[0].shape

(40,)

In [5]:
print(states[0])

[ 1 -1 -1  1 -1  1 -1 -1  1 -1  1  1 -1  1  1 -1  1 -1  1  1  1  1 -1  1
 -1 -1  1  1 -1  1  1  1 -1  1 -1  1  1  1 -1 -1]


In [6]:
def ising_en(ste):
    samples,l=ste.shape
    energy=np.zeros(samples)
    for j in range(samples):
        state1=ste[j]
        for i in range(l):
            energy[j]+=-state1[i]*state1[i-1]
    return energy
        

In [7]:
ising_en(states)

array([ 12., -16.,   4., ...,   0.,  -8.,  -4.])

In [8]:
def ising_energies(states):
    L = states.shape[1]
    J = np.zeros((L, L),)
    for i in range(L): 
        J[i,(i+1)%L]=-1.0
        
    E = np.einsum('...i,ij,...j->...',states,J,states)

    return E

In [9]:
ising_energies(states)

array([ 12., -16.,   4., ...,   0.,  -8.,  -4.])